### Importing necessary libraries

In [2]:
import glob
import os
import re
import c3d
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

### Locating c3d data and sorting by file name

In [3]:
# Change to correct folder location

path = r"C:\Users\davin\OneDrive\Desktop\Bird Data\alldata\Screen Obstacle Flights\20240523_DT_newcameras\20240523_DT_newcameras_tests"


In [5]:
# Bird name to identifier mapping
bird_id_map = {
    "Dartagnan": "01",
    "Charmander": "02",
    "Toothless": "03",
    "Drogon": "04"
}

# Initialize an empty pandas DataFrame
all_c3d_filenames = sorted(glob.glob(path + r"\*\*.c3d"))
c3d_df = pd.DataFrame(columns=["Path", "Bird", "Bird_ID", "Date", "Filename", "Trial"])

# Processing the file paths to extract bird name, date, and trial number
for i, filepath in enumerate(all_c3d_filenames):
    filename = os.path.basename(filepath)  # Extracting filename from the full path
    date = filename[:8]  # Date = First 8 characters of the filename (YYYYMMDD)
    
    # Extract the bird name from the folder path (e.g., Dartagnan, Charmander, Toothless, Drogon)
    bird = None
    bird_id = None
    for bb in bird_id_map.keys():
        if bb in filepath:
            bird = bb
            bird_id = bird_id_map[bb]
            break
    
    # Extract the trial number (last two digits before the ".c3d" extension)
    trial_match = re.search(r'(\d{2})(?=\.c3d$)', filename)
    trial_number = trial_match.group(1) if trial_match else None

    # Create a row with the extracted information including Bird_ID and Trial
    c3d_row = pd.DataFrame({"Path": filepath, 
                            "Bird": bird, 
                            "Bird_ID": bird_id, 
                            "Date": date, 
                            "Filename": filename,
                            "Trial": trial_number}, index=[0])
    
    # Append the row to the DataFrame
    c3d_df = pd.concat([c3d_df, c3d_row], axis=0, ignore_index=True)


### Load C3D data into pandas 

In [7]:
# rows_list = []

# # iterate over all C3D files (in C3D dataframe)
# for index, row in c3d_df.iterrows():    

#     filepath = c3d_df.at[index, 'Path']  # get path to .c3d file
#     bird = c3d_df.at[index, 'Bird']  # get bird name
#     bird_id = c3d_df.at[index, 'Bird_ID']  # get bird ID
#     trial_number = c3d_df.at[index, 'Trial']  # get trial number
#     date = c3d_df.at[index, 'Date']  # get date (YYYYMMDD)
    
#     # Construct the Flight_ID as "Date_BirdID_TrialNumber"
#     flight_id = f"{date}_{bird_id}_{trial_number}"
    
#     # Read the C3D file
#     c3dfile = c3d.Reader(open(filepath, 'rb'))

#     markers = list(range(44))  # markers range from [0, 1, ..., 43]
#     for marker in markers:
#         for f, points, analog in c3dfile.read_frames():
#             if marker < points.shape[0]:

#                 t = float(f) / 120  # Assuming frame rate is 120Hz
#                 X = points[marker, 0]
#                 Y = points[marker, 1]
#                 Z = points[marker, 2]

#                 # Replace zero coordinates with NaN
#                 if (X == 0 or Y == 0 or Z == 0):
#                     X, Y, Z = 'nan', 'nan', 'nan'

#                 # Create a row with the required columns
#                 data_row = {
#                     'Flight_ID': flight_id,  # Add Flight_ID as the first column
#                     'Bird': bird,
#                     'Bird_ID': bird_id,
#                     'Trial': trial_number,
#                     'Marker': marker,
#                     'Time': t,
#                     'Frame': f,
#                     'X': X,
#                     'Y': Y,
#                     'Z': Z
#                 }
#                 # Add new row to the list of dictionaries
#                 rows_list.append(data_row)

# # Convert the list of dictionaries to a pandas DataFrame
# trajectories_df = pd.DataFrame(rows_list)


# trajectories_df

import pandas as pd
import c3d
import os

# Specify the directory where CSV files will be saved
output_dir = r"C:\Users\davin\OneDrive\Desktop\Bird Data\alldata\Screen Obstacle Flights\20240523_DT_newcameras\20240523_DT_newcameras_tests"  # Replace with your desired path
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

# Initialize a dictionary to store data rows for each date
date_data = {}

# Iterate over all C3D files in the C3D dataframe
for index, row in c3d_df.iterrows():    
    filepath = c3d_df.at[index, 'Path']  # Path to .c3d file
    bird = c3d_df.at[index, 'Bird']  # Bird name
    bird_id = c3d_df.at[index, 'Bird_ID']  # Bird ID
    trial_number = c3d_df.at[index, 'Trial']  # Trial number
    date = c3d_df.at[index, 'Date']  # Date (YYYYMMDD)
    
    # Construct Flight_ID
    flight_id = f"{date}_{bird_id}_{trial_number}"
    
    # Read the C3D file
    c3dfile = c3d.Reader(open(filepath, 'rb'))

    markers = list(range(44))  # Markers range from [0, 1, ..., 43]
    for marker in markers:
        for f, points, analog in c3dfile.read_frames():
            if marker < points.shape[0]:
                t = float(f) / 120  # Assuming frame rate is 120Hz
                X = points[marker, 0]
                Y = points[marker, 1]
                Z = points[marker, 2]

                # Replace zero coordinates with NaN
                if X == 0 or Y == 0 or Z == 0:
                    X, Y, Z = 'nan', 'nan', 'nan'

                # Prepare the data row
                data_row = {
                    'Flight_ID': flight_id,
                    'Bird': bird,
                    'Bird_ID': bird_id,
                    'Trial': trial_number,
                    'Marker': marker,
                    'Time': t,
                    'Frame': f,
                    'X': X,
                    'Y': Y,
                    'Z': Z
                }

                # Add the row to the corresponding date's list
                if date not in date_data:
                    date_data[date] = []
                date_data[date].append(data_row)

# Convert each date's list of rows to a DataFrame and save to a CSV file in the specified directory
for date, rows in date_data.items():
    # Create a DataFrame for the specific date
    date_df = pd.DataFrame(rows)
    
    # Define the CSV file name with the specified directory
    filename = os.path.join(output_dir, f"{date}_flights.csv")
    
    # Save the DataFrame to a CSV file
    date_df.to_csv(filename, index=True)




In [8]:
display(date_df)





,Flight_ID,Bird,Bird_ID,Trial,Marker,Time,Frame,X,Y,Z
0,20240614_03_01,Toothless,03,01,0,0.008333,1,745.478394,-1398.27771,140.982437
1,20240614_03_01,Toothless,03,01,0,0.016667,2,745.466003,-1398.275757,140.981888
2,20240614_03_01,Toothless,03,01,0,0.025000,3,745.457397,-1398.28064,140.972168
3,20240614_03_01,Toothless,03,01,0,0.033333,4,745.439575,-1398.294922,140.965576
4,20240614_03_01,Toothless,03,01,0,0.041667,5,745.45343,-1398.307861,140.963577
...,...,...,...,...,...,...,...,...,...,...
1649790,20240614_03_63,Toothless,03,63,30,2.633333,316,nan,nan,nan
1649791,20240614_03_63,Toothless,03,63,30,2.641667,317,nan,nan,nan
1649792,20240614_03_63,Toothless,03,63,30,2.650000,318,nan,nan,nan
1649793,20240614_03_63,Toothless,03,63,30,2.658333,319,nan,nan,nan
